In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
from pandas import pivot_table

# LOAD SAVED DATA FROM PREVIOUS ENGEENIRING

In [2]:
update1 =pd.read_csv('update1.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
update1= update1.loc[update1['Group'] != 0]

# PIVOT TABLES

In [4]:
month = pivot_table(update1, values=['7_days', '60_days', '1_month', '2_month', '3_month', 'spent_on_pos', 'total_spent', 'Cashback', 'total_earn', 'amount_in_rub' ],
                    index=['cl_id'], aggfunc=np.sum)
month1 = pivot_table(update1, values=['spent_on_pos', 'total_spent', 'Cashback', 'total_earn', 'amount_in_rub' ],
                    index=['cl_id'], aggfunc=lambda Cashback: len(Cashback.unique()))


spent_on_open_date = pivot_table(update1, values='amount_in_rub', 
                    index=['cl_id'], columns=['PERIOD_y'], aggfunc=lambda amount_in_rub: len(amount_in_rub.unique()))


day = pivot_table(update1, values='spent_on_pos', 
                    index=['cl_id'], columns=['Day1'], aggfunc=lambda amount_in_rub: len(amount_in_rub.unique()))



trx = pivot_table(update1, values='amount_in_rub', 
                    index=['cl_id'], columns=['trx_category'], aggfunc=np.mean)



group_mcc = pivot_table(update1, values='amount_in_rub', 
                    index=['cl_id'], columns=['Group'], aggfunc=lambda amount_in_rub: len(amount_in_rub.unique()))


region = pivot_table(update1, values='spent_on_pos', 
                    index=['cl_id'], columns=['region'], aggfunc=lambda amount_in_rub: len(amount_in_rub.unique()))

channel_type = pivot_table(update1, values='spent_on_pos', 
                    index=['cl_id'], columns=['channel_type'], aggfunc=lambda amount_in_rub: len(amount_in_rub.unique()))

channel_type = pivot_table(update1, values='spent_on_pos', 
                    index=['cl_id'], columns=['channel_type'], aggfunc=np.sum)

In [8]:
flags_qwe_ewq = update1.copy()

In [7]:
new1= pd.DataFrame()

In [9]:
new1['cl_id']=flags_qwe_ewq['cl_id']
new1['target_flag']=flags_qwe_ewq['target_flag']
new1['target_sum']=flags_qwe_ewq['target_sum']
new1['total_usage']=flags_qwe_ewq['qwe']
new1['POS_usage']=flags_qwe_ewq['ewq']

In [10]:
new1=new1.drop_duplicates()

In [11]:
new1=new1.reset_index()

In [ ]:
new1[month.columns] =month
new1[month1.columns] =month1
new1[spent_on_open_date.columns] =spent_on_open_date
new1[day.columns] =day
new1[trx.columns] =trx
new1[group_mcc.columns] =group_mcc
new1[region.columns] =region
new1[channel_type.columns] =channel_type

In [56]:
data_train=new1

In [57]:
data_train=data_train[data_train['target_flag']!='p']

In [61]:
data_train.target_flag.unique()

array([0., 1.])

In [60]:
data_train.target_flag=data_train.target_flag.astype(float)

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# Train Test

In [62]:
from sklearn.model_selection import StratifiedShuffleSplit

random_splitter = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=777)


In [63]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=777)

for train_index, test_index in splitter.split(data_train, data_train.target_flag):
    d_train = data_train.iloc[train_index]
    d_test = data_train.iloc[test_index]
    
    y_train = data_train['target_flag'].iloc[train_index]
    y_test = data_train['target_flag'].iloc[test_index]

In [64]:
data_train=d_train
data_test=d_test

# COUNTS

In [65]:
def get_counts(df_train, df_test, col, target_col):
    category_val_dict = (
        df_train[df_train[target_col] == 1][col].value_counts()
        /
        df_train[col].value_counts()
    ).to_dict()
    df_train[col + '_counts'] = df_train[col].apply(category_val_dict.get)
    df_test[col + '_counts'] = df_test[col].apply(category_val_dict.get)
    return df_train, df_test

In [66]:
columns_to_get_counts = ['total_usage', 'POS_usage', '1_month', '2_month', '3_month', '60_days', '7_days',
  'Asia', 'CIS', 'Cis', 'Russia', 'asia', 'cheap', 'expencive', 'norm']
    
for col_get_prob in columns_to_get_counts:
    data_train, data_test = get_counts(data_train, data_test, col_get_prob, 'target_flag')

# WOE

In [67]:
def get_woe_v1(df_train, df_test, col, target_col):
    all_good = len(df_train[df_train[target_col] == 1][col])
    all_bad = len(df_train[df_train[target_col] == 0][col])
    odds_series = (
        df_train[df_train[target_col] == 1][col].value_counts()
        /
        df_train[df_train[target_col] == 0][col].value_counts()
    )
    odds_series = odds_series / all_good * all_bad
    category_woe_dict = np.log(odds_series).to_dict()
    df_train[col + '_woe'] = df_train[col].apply(category_woe_dict.get)
    df_test[col + '_woe'] = df_test[col].apply(category_woe_dict.get)
    return df_train, df_test

In [68]:
for col_get_prob in columns_to_get_counts:
    data_train, data_test = get_woe_v1(data_train, data_test, col_get_prob, 'target_flag')

In [69]:
y_train=data_train['target_flag']

In [70]:
x_train1=data_train.fillna(0)
x_test1=data_test.fillna(0)
cat_feat = list(x_train1.dtypes[x_train1.dtypes == object].index)
x_train1=x_train1.drop(columns=cat_feat)
x_test1=x_test1.drop(columns=cat_feat)

In [71]:
del x_train1['target_flag']
del x_test1['target_flag']

In [72]:
del x_train1['cl_id']
del x_test1['cl_id']

# Predictions

In [73]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve

x_test1 = pd.DataFrame(StandardScaler().fit_transform(x_test1), columns=x_test1.columns)
x_train1 = pd.DataFrame(StandardScaler().fit_transform(x_train1), columns=x_train1.columns)
del x_train1['index']
del x_test1['index']

# SVM

In [75]:
from sklearn import svm
svm = svm.SVC()
svm.fit(x_train1, y_train) 

y_pred_svm = svm.predict(x_test1)
y_pred_svm_train = svm.predict(x_train1)
roc_auc_train = np.round(roc_auc_score(y_train, y_pred_svm_train), 2)
roc_auc_test = np.round(roc_auc_score(y_test, y_pred_svm), 2)
print("Train: ", roc_auc_train , "Test: ", roc_auc_test)

Train:  1.0 Test:  0.64


# SVM with linear kernel

In [76]:
from sklearn import svm
svm = svm.SVC( kernel='linear')
svm.fit(x_train1, y_train) 

y_pred_svm1 = svm.predict(x_test1)
y_pred_svm_train1 = svm.predict(x_train1)

roc_auc_train1 = np.round(roc_auc_score(y_train, y_pred_svm_train1), 2)
roc_auc_test1 = np.round(roc_auc_score(y_test, y_pred_svm1), 2)
print("Train: ", roc_auc_train1, "Test: ", roc_auc_test1)

Train:  1.0 Test:  0.66


#  Logistic regression

In [77]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression()
clf_lr.fit(x_train1, y_train)
clf_pred = clf_lr.predict(x_test1)
clf_pred1 = clf_lr.predict(x_train1)
roc_auc_train = np.round(roc_auc_score(y_train, clf_pred1), 2)
roc_auc_test = np.round(roc_auc_score(y_test, clf_pred), 2)
print("Train: ", roc_auc_train , "Test: ", roc_auc_test)

Train:  1.0 Test:  0.64
